In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from Levenshtein import distance as levenshtein_distance
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import mean_absolute_error

In [2]:
od = pd.read_csv("pizza_sales/order_details.csv")
o = pd.read_csv("pizza_sales/orders.csv")
pt = pd.read_csv("pizza_sales/pizza_types.csv", encoding= 'unicode_escape') # codec decode error
p = pd.read_csv("pizza_sales/pizzas.csv")

In [3]:
ood = o.set_index('order_id').join(od.set_index('order_id')).reset_index()

In [4]:
ppt = p.set_index('pizza_type_id').join(pt.set_index('pizza_type_id')).reset_index()

In [5]:
df = ood.set_index('pizza_id').join(ppt.set_index('pizza_id')).reset_index().sort_values(by="order_details_id").reset_index(drop=True)

In [6]:
df.describe()

,order_id,order_details_id,quantity,price
count,48620.000000,48620.000000,48620.000000,48620.000000
mean,10701.479761,24310.500000,1.019622,16.494132
std,6180.119770,14035.529381,0.143077,3.621789
min,1.000000,1.000000,1.000000,9.750000
25%,5337.000000,12155.750000,1.000000,12.750000
50%,10682.500000,24310.500000,1.000000,16.500000
75%,16100.000000,36465.250000,1.000000,20.250000
max,21350.000000,48620.000000,4.000000,35.950000


In [7]:
from pandas_profiling import ProfileReport
prof = ProfileReport(df)
prof.to_file(output_file='pizzaData.html')

/var/folders/58/m9393pr55dd73xt9k6s7jnlm0000gn/T/ipykernel_71987/72503888.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/benjaminpfeffer/no/envs/mlp/lib/python3.8/site-packages/multimethod/__init__.py:315: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  return func(*args, **kwargs)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
groups = df.groupby("order_id").agg({"pizza_type_id":"count"}).reset_index()
multiOrders = groups[groups.pizza_type_id>1].order_id.values

In [9]:
# multi-order dataframe
moDf = df[df.order_id.isin(multiOrders)].reset_index(drop=True)

## Ways to recommend:
### 1) User Similarity (similar prices and times)
### 2) Association (similar orders)
### 3) Item-item similarity * noted as best way from class notes
### 4) Popularity

### From class notes, best recommendation type is item to item
### In our case, this means pizzas that go with pizzas, not order

### First: trying simple co-occurence of pizza type using cosine similarity

In [10]:
simple = moDf[["order_id", "quantity", "pizza_type_id"]]
copyDf = simple.copy()
#copyDf = copyDf[copyDf.quantity==1]
copyDf['quantity'] = 1

In [11]:
copyDf = copyDf.drop_duplicates().reset_index(drop=True)

In [12]:
itemitem = copyDf.pivot(index='pizza_type_id', columns='order_id', values='quantity').fillna(0)

### Use training, validation and testing set

In [13]:
ii = itemitem.transpose().reset_index()

In [14]:
map_pizza_test = itemitem.index.to_list()

In [15]:
pizza_types = itemitem.index.to_list()

In [36]:
# 100 random splits, independent on previous splits
plot_jacs_val = []
plot_cos_val = []
plot_lev_val = []
plot_rmse_val = []
plot_mae_val = []

plot_jacs = []
plot_cos = []
plot_lev = []
plot_rmse = []
plot_mae = []

for i in range(100):
    print(i)
    # Select the random sampled training set
    train_size = 0.70
    val_size = 0.15
    ii_train = ii.sample(n=int(np.floor(len(ii)*train_size)))
    remain = ii[~ii['order_id'].isin(ii_train["order_id"])]
    ii_val = remain.sample(n=int(np.floor(len(remain)*val_size)))
    ii_test = remain[~remain['order_id'].isin(ii_val["order_id"])]
    
    
    ii_train = ii_train.set_index('order_id').transpose()
    ii_val = ii_val.set_index('order_id').transpose()
    ii_test = ii_test.set_index('order_id').transpose()

    # Fit the nearest neighbors model using cosine similarity on the training set
    knn = NearestNeighbors(metric='cosine', algorithm='brute')
    knn.fit(ii_train.values)
    distances_train, indices_train = knn.kneighbors(ii_train.values, n_neighbors=len(ii_train))


    # Make n recommendations and calculate metrics
    ns = np.arange(1,15)
    all_mins = [] # store the minimum similarities
    outer_jacs_val = []
    outer_cos_val = []
    outer_lev_val = []
    outer_rmse_val = []
    outer_mae_val = []
    outer_jacs = []
    outer_cos = []
    outer_lev = []
    outer_rmse = []
    outer_mae = []
    for n in ns:
        #all_precs = []
        all_jacs_val = [] # jaccard similarity
        all_cos_val = [] # cosine similarity
        all_lev_val = [] # lev distance = edit distance
        all_rmse_val = [] # RMSE
        all_mae_val = [] # mae
        all_jacs = [] # jaccard similarity
        all_cos = [] # cosine similarity
        all_lev = [] # lev distance = edit distance
        all_rmse = [] # RMSE
        all_mae = [] # mae
        idx = 0
        for i in pizza_types:
            bbq_ckn_val = ii_val.transpose()[ii_val.transpose()[i]==1]
            bbq_ckn = ii_test.transpose()[ii_test.transpose()[i]==1]

            # Find the counts
            counts_val = []
            for i in bbq_ckn_val.columns:
                counts_val.append(sum(bbq_ckn_val[i]))
            counts = []
            for i in bbq_ckn.columns:
                counts.append(sum(bbq_ckn[i]))

            # Create dataframe
            result_df_val = pd.DataFrame()
            result_df_val["Pizza"] = bbq_ckn_val.columns
            result_df_val["Count"] = counts_val
            # Create dataframe
            result_df = pd.DataFrame()
            result_df["Pizza"] = bbq_ckn.columns
            result_df["Count"] = counts

            # Iterate through different pizzas
            curr = indices_train[idx]
            dist = distances_train[idx]
            idx += 1

            # Extract predicted vs true
            recPizzas = [map_pizza_test[j] for j in curr[1:n+1]]
            truePizzas_val = list(result_df_val.sort_values("Count", ascending=False).Pizza.iloc[:n].values)
            truePizzas = list(result_df.sort_values("Count", ascending=False).Pizza.iloc[:n].values)

            # Get jaccard similarity
            correct_val = [i for i in recPizzas if i in truePizzas_val]
            combined = recPizzas.copy()
            combined.extend(truePizzas_val)
            union_val = set(combined)
            jac = len(correct_val)/len(union_val)
            all_jacs_val.append(jac)
            
            correct = [i for i in recPizzas if i in truePizzas]
            combined = recPizzas.copy()
            combined.extend(truePizzas)
            union = set(combined)
            jac = len(correct)/len(union)
            all_jacs.append(jac)

            
            # Get cosine similarity of results
            # Processing assistance found: https://stackoverflow.com/questions/28819272/python-how-to-calculate-the-cosine-similarity-of-two-word-lists
            # count word occurrences
            a_vals = Counter(recPizzas)
            b_vals_val = Counter(truePizzas_val)
            b_vals = Counter(truePizzas)

            # convert to word-vectors - val
            words  = list(a_vals.keys() | b_vals_val.keys())
            a_vect = [a_vals.get(word, 0) for word in words]
            b_vect = [b_vals_val.get(word, 0) for word in words]  
            cos = cosine_similarity(np.array(a_vect).reshape(1, -1), np.array(b_vect).reshape(1, -1))
            all_cos_val.append(cos)
            
            # test
            words  = list(a_vals.keys() | b_vals.keys())
            a_vect = [a_vals.get(word, 0) for word in words]
            b_vect = [b_vals.get(word, 0) for word in words]  
            cos = cosine_similarity(np.array(a_vect).reshape(1, -1), np.array(b_vect).reshape(1, -1))
            all_cos.append(cos)

            # Levenshtein distance
            all_lev_val.append(levenshtein_distance(truePizzas_val, recPizzas))
            all_lev.append(levenshtein_distance(truePizzas, recPizzas))

            # Ranked rmse
            allTruePizzas = list(result_df_val.sort_values("Count", ascending=False).Pizza.values)
            compareDf_val = pd.DataFrame()
            compareDf_val["Pizza"] = list(union_val)
            rec_ranks = []
            true_ranks = []
            for pizza in union_val:
                try:
                    rec_ranks.append(recPizzas.index(pizza)+1)
                except:
                    rec_ranks.append(np.nan)
                try:
                    true_ranks.append(allTruePizzas.index(pizza))
                except:
                    true_ranks.append(np.nan)
            compareDf_val["RecRank"] = rec_ranks
            compareDf_val["TrueRank"] = true_ranks
            compareDf_val = compareDf_val.dropna()
            rmse = sqrt(mean_squared_error(compareDf_val.TrueRank, compareDf_val.RecRank))
            all_rmse_val.append(rmse)
            
            
            allTruePizzas = list(result_df.sort_values("Count", ascending=False).Pizza.values)
            compareDf = pd.DataFrame()
            compareDf["Pizza"] = list(union)
            rec_ranks = []
            true_ranks = []
            for pizza in union:
                try:
                    rec_ranks.append(recPizzas.index(pizza)+1)
                except:
                    rec_ranks.append(np.nan)
                try:
                    true_ranks.append(allTruePizzas.index(pizza))
                except:
                    true_ranks.append(np.nan)
            compareDf["RecRank"] = rec_ranks
            compareDf["TrueRank"] = true_ranks
            compareDf = compareDf.dropna()
            rmse = sqrt(mean_squared_error(compareDf.TrueRank, compareDf.RecRank))
            all_rmse.append(rmse)
            
            # mae score
            mae = mean_absolute_error(compareDf.TrueRank, compareDf.RecRank)
            all_mae.append(mae)
            mae = mean_absolute_error(compareDf_val.TrueRank, compareDf_val.RecRank)
            all_mae_val.append(mae)

        outer_jacs_val.append(np.array(all_jacs_val).mean())
        outer_cos_val.append(np.array(all_cos_val).mean())
        outer_lev_val.append(np.array(all_lev_val).mean())
        outer_rmse_val.append(np.array(all_rmse_val).mean())
        outer_mae_val.append(np.array(all_mae_val).mean())
        outer_jacs.append(np.array(all_jacs).mean())
        outer_cos.append(np.array(all_cos).mean())
        outer_lev.append(np.array(all_lev).mean())
        outer_rmse.append(np.array(all_rmse).mean())
        outer_mae.append(np.array(all_mae).mean())

    plot_jacs_val.append(np.array([np.array(outer_jacs_val)]))
    plot_cos_val.append(np.array([np.array(outer_cos_val)]))
    plot_lev_val.append(np.array([np.array(outer_lev_val)]))
    plot_rmse_val.append(np.array([np.array(outer_rmse_val)]))
    plot_mae_val.append(np.array([np.array(outer_mae_val)]))
    plot_jacs.append(np.array([np.array(outer_jacs)]))
    plot_cos.append(np.array([np.array(outer_cos)]))
    plot_lev.append(np.array([np.array(outer_lev)]))
    plot_rmse.append(np.array([np.array(outer_rmse)]))
    plot_mae.append(np.array([np.array(outer_mae)]))
    
# Can perform mean of means here because we care about the inner mean value, and want the mean of it.
output_data_jacs_val = np.array(plot_jacs_val).mean(axis=0)
output_data_cos_val = np.array(plot_cos_val).mean(axis=0)
output_data_lev_val = np.array(plot_lev_val).mean(axis=0)
output_data_rmse_val = np.array(plot_rmse_val).mean(axis=0)
output_data_mae_val = np.array(plot_mae_val).mean(axis=0)
output_data_jacs = np.array(plot_jacs).mean(axis=0)
output_data_cos = np.array(plot_cos).mean(axis=0)
output_data_lev = np.array(plot_lev).mean(axis=0)
output_data_rmse = np.array(plot_rmse).mean(axis=0)
output_data_mae = np.array(plot_mae).mean(axis=0)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [37]:
metricDf_val = pd.DataFrame()
metricDf_val["JacSim"] = output_data_jacs_val[0]
metricDf_val["CosSim"] = output_data_cos_val[0]
metricDf_val["LevDis"] = output_data_lev_val[0]
metricDf_val["RMSE"] = output_data_rmse_val[0]
metricDf_val["MAE"] = output_data_mae_val[0]
metricDf_val["NumRec"] = metricDf_val.index + 1

In [25]:
metricDf_val.to_csv("valMetrics.csv")

In [34]:
metricDf = pd.DataFrame()
metricDf["JacSim"] = output_data_jacs[0]
metricDf["CosSim"] = output_data_cos[0]
metricDf["LevDis"] = output_data_lev[0]
metricDf["RMSE"] = output_data_rmse[0]
metricDf["MAE"] = output_data_mae[0]
metricDf["NumRec"] = metricDf.index + 1

In [24]:
metricDf.to_csv("metrics.csv")

In [79]:
# Select where we don't see as drastic of a rise in error = 5 pizzas

### Given a cart, average cosine sim for each item and display the optimal n (5)

In [25]:
# FINAL SYSTEM - USED LIVE AND INCLUDES ALL GIVEN DATA

n=5 # 6 pizza recommendations

knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(itemitem.values)
distances, indices = knn.kneighbors(itemitem.values, n_neighbors=n+1) #+1 for removal of self similarity

map_pizza = itemitem.index.to_list()



In [26]:
cart = ["bbq_ckn", "hawaiian", "five_cheese"]
n = 5
for i,item in enumerate(cart):
    idx = map_pizza_test.index(item)
    currDf = pd.DataFrame()
    currDf["Distance"] = distances[idx]
    currDf["Indices"] = indices[idx]
    currDf = currDf.sort_values(by="Indices")
    if(i==0):
        totalDf = currDf
    else:
        totalDf = totalDf.append(currDf)

distanceDf = totalDf.groupby("Indices").agg({"Distance":"mean"}).sort_values(by="Distance").reset_index() 
all_recs = [(map_pizza_test[i], 1-d) for i,d in zip(distanceDf.Indices,distanceDf.Distance) if map_pizza_test[i] not in cart]
all_recs[:n] # similarity shown

/var/folders/58/m9393pr55dd73xt9k6s7jnlm0000gn/T/ipykernel_70875/2838131429.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  totalDf = totalDf.append(currDf)
/var/folders/58/m9393pr55dd73xt9k6s7jnlm0000gn/T/ipykernel_70875/2838131429.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  totalDf = totalDf.append(currDf)


[('cali_ckn', 0.13534339737958456),
 ('thai_ckn', 0.1306199792665136),
 ('pepperoni', 0.12759280601912637),
 ('southw_ckn', 0.1273504991106329),
 ('classic_dlx', 0.12360246863054258)]